In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [28]:
df_driving_data = pd.read_csv('/content/drive/MyDrive/Fleet Insight Dashboard/Initial Data/driving_data.csv')
df_vehicle_data = pd.read_csv('/content/drive/MyDrive/Fleet Insight Dashboard/Initial Data/vehicle_data.csv')

In [29]:
import numpy as np
import pandas as pd

def fuel_cost(df_vehicle_data, df_driving_data):
    df = pd.merge(df_driving_data[['vid', 'day_id', 'distance_total']],
                  df_vehicle_data[['vid', 'vehicle_class', 'fuel_type', 'drivetrain_type']], how='left', on='vid')

    lst = []
    for index, row in df.iterrows():
        if row['vehicle_class'] == 2:
            lst.append(np.random.uniform(9.1, 12.5))
        elif row['vehicle_class'] == 3:
            lst.append(np.random.uniform(8.5, 11.5))
        elif row['vehicle_class'] == 4:
            lst.append(np.random.uniform(8.1, 10.5))
        elif row['vehicle_class'] == 5:
            lst.append(np.random.uniform(7.2, 9.5))
        elif row['vehicle_class'] == 6:
            lst.append(np.random.uniform(6.5, 8.7))
        elif row['vehicle_class'] == 7:
            lst.append(np.random.uniform(5.7, 7.9))
        else:
            lst.append(np.random.uniform(4.9, 7.1))

    df_driving_data['average_consumption'] = lst
    df_driving_data['fuel_cost'] = round((2.39 / df_driving_data['average_consumption'] * df_driving_data['distance_total']), 2)
    return df_driving_data

def cost_per_vehicle(df_vehicle_data, df_driving_data):
    vehicle_cost_data = pd.DataFrame()
    sample = pd.DataFrame()

    vehicle_cost_data['vid'] = df_vehicle_data['vid'].copy()
    sample['vid'] = df_vehicle_data['vid'].copy()

    df_driving_data['month'] = ''  # Fix: use empty string instead of integer

    df_driving_data.loc[(df_driving_data['day_id'] > 0) & (df_driving_data['day_id'] <= 4), ['month']] = 'jan'
    df_driving_data.loc[(df_driving_data['day_id'] > 4) & (df_driving_data['day_id'] <= 8), ['month']] = 'feb'
    df_driving_data.loc[(df_driving_data['day_id'] > 8) & (df_driving_data['day_id'] <= 12), ['month']] = 'mar'
    df_driving_data.loc[(df_driving_data['day_id'] > 12) & (df_driving_data['day_id'] <= 16), ['month']] = 'apr'
    df_driving_data.loc[(df_driving_data['day_id'] > 16) & (df_driving_data['day_id'] <= 20), ['month']] = 'may'
    df_driving_data.loc[(df_driving_data['day_id'] > 20) & (df_driving_data['day_id'] <= 30), ['month']] = 'jun'
    df_driving_data.loc[(df_driving_data['day_id'] > 30), ['month']] = 'jul'

    grouped = df_driving_data.groupby(df_driving_data.month)
    jan = grouped.get_group('jan')
    feb = grouped.get_group('feb')
    mar = grouped.get_group('mar')
    apr = grouped.get_group('apr')
    may = grouped.get_group('may')
    jun = grouped.get_group('jun')
    jul = grouped.get_group('jul')

    arr = [jan, feb, mar, apr, may, jun, jul]
    arr_name = ['January', 'February', 'March', 'April', 'May', 'June', 'July']

    i = 0
    sum = {}
    for month in arr:
        for index, row in month.iterrows():
            if int(row['vid']) in sum.keys():
                sum[int(row['vid'])] += row['fuel_cost']
            else:
                sum[int(row['vid'])] = row['fuel_cost']
        sample['month'] = arr_name[i]
        sample['fuel_cost_total'] = sample['vid'].map(sum)
        i += 1
        vehicle_cost_data = pd.concat([vehicle_cost_data, sample], ignore_index=True)  # Fix: replace append

    vehicle_cost_data.fillna(0, inplace=True)
    vehicle_cost_data['insurance_cost'] = np.around(np.random.normal(200, 20, size=len(vehicle_cost_data)), decimals=2)
    vehicle_cost_data['maintenance_cost'] = np.around(np.random.normal(1200, 100, size=len(vehicle_cost_data)), decimals=2)
    vehicle_cost_data['total_cost'] = vehicle_cost_data['fuel_cost_total'] + vehicle_cost_data['insurance_cost'] + vehicle_cost_data['maintenance_cost']
    return vehicle_cost_data

def vehicle_build_year(df):
    df['vehicle_construction_year'] = np.random.randint(2000, 2018, size=len(df))
    return df

def maintenance_start_value(df):
    df['scheduled_maintenance'] = np.random.randint(0, 22, size=len(df))
    return df

def accident_probability(df_vehicle_data):
    df_vehicle_data['accident_probability'] = np.random.randint(0, 100, size=len(df_vehicle_data))
    return df_vehicle_data

def vehicle_capacity(df_vehicle_data):
    lst = []
    for index, row in df_vehicle_data.iterrows():
        if row['vehicle_class'] == 2:
            lst.append(np.random.randint(1500, 3000))
        elif row['vehicle_class'] == 3:
            lst.append(np.random.randint(3000, 5500))
        elif row['vehicle_class'] == 4:
            lst.append(np.random.randint(5500, 7000))
        elif row['vehicle_class'] == 5:
            lst.append(np.random.randint(7000, 9500))
        elif row['vehicle_class'] == 6:
            lst.append(np.random.randint(9500, 13000))
        elif row['vehicle_class'] == 7:
            lst.append(np.random.randint(13000, 19000))
        else:
            lst.append(np.random.randint(19000, 60000))
    df_vehicle_data['load_capacity'] = lst
    return df_vehicle_data

def vehicle_position(df_vehicle_data):
    df = pd.read_csv('/content/drive/MyDrive/Fleet Insight Dashboard/Initial Data/random-locations.csv')
    df_vehicle_data['position_latitude'] = df['latitude'].copy()
    df_vehicle_data['position_longitude'] = df['longitude'].copy()
    return df_vehicle_data

def vehicle_status(df_vehicle_data):
    status = np.random.choice(['accident', 'unused', 'idle', 'on time', 'delayed'], p=[0.05, 0.1, 0.15, 0.4, 0.3],
                              size=len(df_vehicle_data))
    df_vehicle_data['vehicle_status'] = status
    df_vehicle_data.loc[df_vehicle_data['scheduled_maintenance'] == 0, ['vehicle_status']] = 'maintenance'
    return df_vehicle_data

def generate_licence_plate(df_vehicle_data):
    lst = []
    for i in range(len(df_vehicle_data)):
        i += 1
        if i <= 9:
            lst.append(('NRL-00' + str(i)))
        elif i <= 99:
            lst.append(('NRL-0' + str(i)))
        else:
            lst.append(('NRL-' + str(i)))
    df_vehicle_data['licence_plate'] = lst
    return df_vehicle_data


In [30]:
# Step 1: Calculate fuel consumption and cost
df_driving_data_final = fuel_cost(df_vehicle_data, df_driving_data)

# Step 2: Compute monthly fuel, insurance, and maintenance costs
df_vehicle_costs = cost_per_vehicle(df_vehicle_data, df_driving_data)

# Step 3: Add build year
df_vehicle_data_alpha = vehicle_build_year(df_vehicle_data)

# Step 4: Add scheduled maintenance value
df_vehicle_data_beta = maintenance_start_value(df_vehicle_data_alpha)

# Step 5: Add accident probability score
df_vehicle_data_gamma = accident_probability(df_vehicle_data_beta)

# Step 6: Add vehicle load capacity based on class
df_vehicle_data_delta = vehicle_capacity(df_vehicle_data_gamma)

# Step 7: Add geolocation position (lat/lon)
df_vehicle_data_theta = vehicle_position(df_vehicle_data_delta)

# Step 8: Determine vehicle operational status
df_vehicle_data_prelim = vehicle_status(df_vehicle_data_theta)

# Step 9: Generate license plates
df_vehicle_data_final = generate_licence_plate(df_vehicle_data_prelim)


In [31]:
df_driving_data_final

,Unnamed: 0,vid,pid,day_id,distance_total,speed_data_duration_hrs_includes_zero,driving_time_seconds_no_zero,max_speed,total_average_speed_includes_zero,total_median_speed_includes_zero,...,spd_cat_3_distance,spd_cat_4_distance,spd_cat_5_distance,average_consumption,fuel_cost,tire_sensor,engine_sensor,break_sensor,maintenance_need,month
0,0,37,3,14,7.447608,0.521667,1072.0,49.051530,14.276564,5.593784,...,0.000000,0.000000,7.447608,5.865229,3.03,0,0,0,0,apr
1,1,10,3,1,67.535063,3.880000,6316.0,51.278274,8.819466,0.000000,...,1.750527,9.157190,23.148930,8.278656,19.50,0,0,0,0,jan
2,2,10,3,2,66.972544,4.054444,6307.0,79.153317,8.426972,0.000000,...,6.657324,6.503542,21.005822,8.402568,19.05,1,1,0,1,jan
3,3,10,3,3,64.851790,4.081111,6283.0,47.623214,8.234758,0.000000,...,2.241936,5.839693,25.525333,8.768011,17.68,0,0,0,0,jan
4,4,10,3,6,72.198676,4.131667,6663.0,65.738244,9.018202,0.000000,...,6.361258,7.838128,23.060817,10.220942,16.88,0,0,0,0,feb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2984,2984,239,17,129,124.780116,3.502778,9876.0,74.393476,35.623189,42.978763,...,8.991962,58.212552,0.199638,7.118798,41.89,1,0,1,1,jul
2985,2985,239,17,130,109.725524,4.027222,9022.0,74.680694,27.246019,20.056359,...,21.967134,24.788408,4.626673,6.523900,40.20,1,0,1,1,jul
2986,2986,239,17,131,214.484149,4.801667,15544.0,76.746390,44.668688,50.579755,...,57.531506,10.245859,15.014577,6.516686,78.66,1,0,0,0,jul
2987,2987,239,17,132,136.145375,3.345278,10274.0,74.766013,40.697779,45.527099,...,1.026913,23.051028,1.897652,6.010101,54.14,1,0,1,1,jul


In [32]:
df_vehicle_data_final

,Unnamed: 0,vid,vehicle_class,vocation,vehicle_type,fuel_type,drivetrain_type,vehicle_construction_year,load_capacity,scheduled_maintenance,position_latitude,position_longitude,vehicle_status,licence_plate,accident_probability,predicted_maintenance_probability,predicted_weeks_until_maintenance
0,0,1,7,Beverage Delivery,Tractor,Diesel,Conventional,2015,16678,19,49.642101,-89.671216,on time,NRL-001,0,0.022588,30
1,1,2,7,Beverage Delivery,Tractor,Diesel,Conventional,2011,16448,17,44.769769,-84.360045,on time,NRL-002,3,0.022588,30
2,2,3,8,Beverage Delivery,Tractor,Diesel,Parallel Hybrid,2013,48423,20,40.068301,-83.307116,delayed,NRL-003,86,0.022588,30
3,3,4,8,Beverage Delivery,Tractor,Diesel,Parallel Hybrid,2005,30879,19,37.132773,-88.878560,delayed,NRL-004,67,0.022588,30
4,4,5,8,Beverage Delivery,Tractor,Diesel,Conventional,2002,23839,5,33.941412,-86.963802,delayed,NRL-005,81,0.022588,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,219,575,3,Food Delivery,Straight Truck,Diesel,Conventional,2007,3259,12,43.210854,-97.148083,unused,NRL-220,99,0.022588,30
220,220,576,4,Parcel Delivery,Walk In,Diesel,Parallel Hybrid,2005,5967,10,37.509799,-113.958857,on time,NRL-221,11,0.041027,30
221,221,577,4,Parcel Delivery,Walk In,Diesel,Parallel Hybrid,2010,6395,9,45.566924,-93.004176,delayed,NRL-222,1,0.022588,30
222,222,578,4,Parcel Delivery,Walk In,Diesel,Conventional,2000,5971,15,34.380310,-94.986915,on time,NRL-223,26,0.022588,30


In [33]:
df_driving_data_final.to_csv('/content/drive/MyDrive/Fleet Insight Dashboard/Cleaned Data/driving_data_final.csv', index=False)
df_vehicle_data_final.to_csv('/content/drive/MyDrive/Fleet Insight Dashboard/Cleaned Data/vehicle_data_final.csv', index=False)